In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from matplotlib import pyplot as plt

from dist import RawJointDist, CPT
from rv import Variable, binvar, Unit
import numpy as np
from pdg import PDG
np.set_printoptions(precision=3)

In [4]:
from lib import square